<a href="https://colab.research.google.com/github/gbouras13/phold/blob/main/run_phold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Phold

[phold](https://github.com/gbouras13/phold) is a sensitive annotation tool for bacteriophage genomes and metagenomes using protein structural homology.

phold uses the [ProstT5](https://github.com/mheinzinger/ProstT5) protein language model to translate protein amino acid sequences to the 3Di token alphabet used by [Foldseek](https://github.com/steineggerlab/foldseek). Foldseek is then used to search these against a database of 803k protein structures mostly predicted using [Colabfold](https://github.com/sokrypton/ColabFold).


* Please make sure you change the runtime to T4 GPU, as this will make Phold run faster.
* To do this, go to the top toolbar, then to Runtime -> Change runtime type -> Hardware accelerator
* To run the cells, press the play button on the left side
* Cells 1 and 2 install phold and download the database
* Once they have been run, you can re-run Cell 3 as many times as you would like



In [1]:
#@title 1. Install phold

#@markdown This cell installs Phold. It will take a few minutes. Please be patient

%%time
import os
from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"
PYTHON_VERSION = python_version


if not os.path.isfile("MAMBA_READY"):
  print("installing mamba...")
  os.system("wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh")
  os.system("bash Mambaforge-Linux-x86_64.sh -bfp /usr/local")
  os.system("mamba config --set auto_update_conda false")
  os.system("touch MAMBA_READY")

if not os.path.isfile("PHOLD_READY"):
  print("installing phold...")
  os.system(f"mamba install -y -c conda-forge -c bioconda phold==0.1.2 python='{PYTHON_VERSION}'  pytorch=*=cuda* ")
  os.system("touch PHOLD_READY")



installing mamba...
installing phold...
CPU times: user 611 ms, sys: 68.2 ms, total: 679 ms
Wall time: 2min 39s


In [3]:
#@title 2. Download phold databases

#@markdown This cell downloads the phold database. It will take a few minutes. Please be patient.


%%time
print("Downloading phold database. This will take a few minutes. Please be patient :)")
os.system("phold install")




installing phold database
CPU times: user 687 ms, sys: 114 ms, total: 801 ms
Wall time: 3min 34s


0

In [16]:
#@title 3. Run Phold

#@markdown Upload your phage(s) as a Pharokka GenBank or a nucleotide input FASTA

#@markdown Click on the folder icon to the left and use file upload button.

#@markdown Then provide a directory for phold's output.
#@markdown The default is 'output_phold'.

#@markdown You can click FORCE to overwrite the output directory.
#@markdown This may be useful if your earlier phold run has crashed for whatever reason.

#@markdown The results of Phold will be in the folder icon on the left hand panel.
#@markdown Additionally, it will be zipped so you can download the whole directory.

#@markdown The file to download is PREFIX.zip, where PREFIX is what you provided

#@markdown If you do not see the output directory,
#@markdown refresh the window by either clicking the folder with the refresh icon below "Files"
#@markdown or double click and select "Refresh".



import os
import subprocess
import zipfile
INPUT_FILE = '' #@param {type:"string"}
OUT_DIR = ''  #@param {type:"string"}
PREFIX = ''  #@param {type:"string"}
FORCE = False  #@param {type:"boolean"}
SEPARATE = False  #@param {type:"boolean"}

# Construct the command
command = f"phold run -i {INPUT_FILE} -t 4 -o {OUT_DIR} -p {PREFIX}"

if FORCE is True:
  command = f"{command} -f"
if SEPARATE is True:
  command = f"{command} --separate"


# Execute the command
try:
    print("Running phold")
    subprocess.run(command, shell=True, check=True)
    print("phold completed successfully.")
    print(f"Your output is in {OUT_DIR}.")
    print(f"Zipping the output directory so you can download it all in one go.")

    zip_filename = f"{OUT_DIR}.zip"

    # Zip the contents of the output directory
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(OUT_DIR):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), OUT_DIR))
    print(f"Output directory has been zipped to {zip_filename}")


except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e}")







Running phold
phold completed successfully.
Your output is in output_phold_combined.
Zipping the output directory so you can download it all in one go.
Output directory has been zipped to output_phold_combined.zip
